# Extract observables transmissibility

In [ ]:
import json
import pandas as pd
from os.path import join
import os

from scseirx import analysis_functions as af

# for progress bars
from ipywidgets import IntProgress
from IPython.display import display

## School configurations

In [ ]:
# characteristics (# classes, # students / class, # teachers) of the "average" 
# school, depending on school type. These characteristics were determined in 
# interviews with Austrian teachers and from statistics about Austrian schools 
# (year 2017/18, page 10: https://www.bmbwf.gv.at/Themen/schule/schulsystem/gd.html)
# NOTE: "students" indicates the number of students per class
with open('params/sensitivity_analysis_school_characteristics.json', 'r') as fp:
    school_characteristics = json.load(fp)

In [27]:
school_types = ['primary', 'primary_dc', 'lower_secondary', 'lower_secondary_dc',
                'upper_secondary', 'secondary']
transmission_risks = [1.5]

school_configs = [(stype, school_characteristics[stype]['classes'], \
                  school_characteristics[stype]['students'], trisk) \
                  for stype in school_types\
                  for trisk in transmission_risks]

## Increased transmission risk

### Measures

In [ ]:
## parameter ranges 
# student and teacher streening intervals (in days)
s_screen_range = [None, 3, 7]
t_screen_range = [None, 3, 7]
# test technologies (and test result turnover times) used in the
# different scenarios
test_types = ['same_day_antigen']
# specifies whether the index case will be introduced via an
# employee or a resident
index_cases = ['student', 'teacher']
# specifies whether teachers wear masks
student_masks = [True, False]
teacher_masks = [True, False]
half_classes = [True, False]
transmission_risk_ventilation_modifiers = [1, 0.36]

params = [(i, j, k, l, m, n, o, p)\
              for i in test_types \
              for j in index_cases \
              for k in s_screen_range \
              for l in t_screen_range \
              for m in student_masks \
              for n in teacher_masks \
              for o in half_classes \
              for p in transmission_risk_ventilation_modifiers]

### Extraction

In [ ]:
turnovers = {'same':0, 'one':1, 'two':2, 'three':3}
bmap = {True:'T', False:'F'}
res_path = '../../data/sensitivity_analysis/simulation_results/transmissibility' 

f = IntProgress(min=0, max=len(params) * len(school_configs))
display(f)
c = 0 # counter for progress bar
for school_type, N_classes, class_size, trisk in school_configs:
    school_string = '{}_classes-{}_students-{}'.format(school_type, N_classes, class_size)
    school = {'type':school_type, 'classes':N_classes,
              'students':class_size}
    
    stype = school['type']
    sname = '{}_classes-{}_students-{}'.format(\
        stype, school['classes'], school['students'])

    spath_ensmbl = join(res_path,'/{}/{}/ensembles/{}'\
                        .format(stype, trisk, school_string))
    spath_obs = join(res_path,'{}/{}/observables'\
                     .format(stype, trisk))
    
    try:
        os.mkdir(spath_obs)
    except FileExistsError:
        pass 
    
    observables = pd.DataFrame()
    for ttype, index_case, s_screen_interval, t_screen_interval, student_mask, \
                teacher_mask, half_classes, ventilation_mod in params:
        
        turnover, _, test = ttype.split('_')
        turnover = turnovers[turnover]
        
        measure_string = '{}_test-{}_turnover-{}_index-{}_tf-{}_sf-{}_tmask-{}'\
        .format(stype, test, turnover, index_case[0], t_screen_interval,
                s_screen_interval, bmap[teacher_mask]) +\
                '_smask-{}_half-{}_vent-{}_trisk-{}'\
        .format(bmap[student_mask], bmap[half_classes], ventilation_mod, trisk)
        
        ensmbl = pd.read_csv(join(spath_ensmbl, measure_string + '.csv'))
        
        row = {'transmission_risk_modifier':trisk,
               'test_type':test,
               'turnover':turnover,
               'index_case':index_case,
               'student_screen_interval':s_screen_interval,
               'teacher_screen_interval':t_screen_interval,
               'student_mask':student_mask,
               'teacher_mask':teacher_mask,
               'half_classes':half_classes,
               'ventilation_modification':ventilation_mod}
        
        ensmbl = ensmbl[ensmbl['infected_agents'] > 0]
        for col in ensmbl.columns:
            row.update(af.get_statistics(ensmbl, col))
        observables = observables.append(row, ignore_index=True)

        f.value = c # update the progress bar
        c += 1
    # calculate the number of tests per day and agent in the school
    observables['N_tests_per_day_per_agent'] = \
    (observables['N_diagnostic_tests_median'] + observables['N_preventive_tests_median']) /\
    observables['duration_median'] / observables['N_school_agents_median']
    
    screen_cols = ['transmission_risk_modifier','test_type', 'turnover',
                   'index_case', 'student_screen_interval',
            'teacher_screen_interval', 'student_mask', 'teacher_mask',
            'half_classes', 'ventilation_modification']

    other_cols = [c for c in observables.columns if c not in screen_cols]
    observables = observables[screen_cols + other_cols]
    
    for col in ['infected_teachers_median', 'infected_students_median', 
                        'infected_family_members_median',
                        'infected_teachers_0.90', 'infected_students_0.90', 
                        'infected_family_members_0.90']:
        observables[col] = observables[col].round(0).astype(int)
    
    observables.to_csv(join(spath_obs, school_string + '.csv'), index=False)

### Combine all ensembles

In [29]:
res_path = '../../data/sensitivity_analysis/simulation_results/transmissibility'
observables = pd.DataFrame()
for stype, N_classes, class_size, trisk in school_configs:
    school_string = '{}_classes-{}_students-{}'.format(stype, N_classes, class_size)
    sname = '{}_classes-{}_students-{}'.format(\
        stype, school['classes'], school['students'])
    spath_obs = join(res_path,'{}/{}/observables'\
                     .format(stype, trisk))
    obs = pd.read_csv(join(spath_obs, school_string + '.csv'))
    observables = pd.concat([observables, obs])

observables.to_csv(join(res_path, stype + '_inc_transk_risk.csv'), index=False)

## Worst cases

In [31]:
src_path = '../../data/sensitivity_analysis/simulation_results/worst_case'
for stype in school_types:
    stype_data = af.get_data(stype, src_path, test_participation_rate=True,
            reduced_class_size=True, reduced_mask_efficiency=True,
            transmission_risk_modifier=True)
    stype_data.to_csv(join(src_path, '{}_combined_ensembles.csv').format(stype), index=False)